In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
from lxml import etree as ET
pd.options.display.max_columns=50

In [2]:
def household(household_df,persons,mzone):
    
    household_df=household_df.sort_values(by=['hhno'])
    household_df['hhno']=household_df['hhno'].astype(str)
    
#     persons['hhno']=persons['hhno'].astype(str)
#     persons['pno']=persons['pno'].astype(str)

#     persons['person_id']=persons['hhno']+'-'+persons['pno']

    
    household_w_pop=pd.merge(household_df,persons[['hhno','person_id']],on=['hhno'],how='left')
    
    hh_pop_mzone=pd.merge(household_w_pop,mzone,left_on=['hhparcel'],right_on=['MAZID'],how='left')
    
    hh_pop_mzone=hh_pop_mzone[['hhno','person_id','hhvehs','hhincome','X_COORD','Y_COORD']]
    
    return hh_pop_mzone

In [3]:
def population(household_df,persons):
    
    household_df['hhno']=household_df['hhno'].astype(str)

    
    household_w_pop=pd.merge(household_df,persons[['hhno','person_id','pagey','pgend']],
                             on=['hhno'],how='right')
    
    household_w_pop=household_w_pop[['person_id','hhno','pagey','pgend','hhvehs']]
    
    return household_w_pop

In [4]:
households=pd.read_table('../SF-CHAMP Outputs/_household_2.dat',sep='\t',header='infer')
# person=pd.read_table('../SF-CHAMP Outputs/_person_2.dat',sep='\t',header='infer')
# mzone=pd.read_csv('../SF-CHAMP Outputs/mzone_short.csv')

In [5]:
person_ei=pd.read_csv('../SF_CHAMP_Converted/agent_attribs.csv.gz',compression='gzip')
person_ei.head()

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT
0,63,63-3,female,30,38016,auto_sufficient,7.810582
1,63,63-7,male,9,38016,auto_sufficient,7.389009
2,131,131-2,male,31,29512,auto_deficient,7.402262
3,320,320-1,female,37,108069,auto_deficient,24.262024
4,320,ix-320-2,male,37,108069,auto_deficient,12.795260


In [6]:
len(person_ei['person_id'].unique())

12802

In [5]:
#trip=pd.read_table('../SF-CHAMP Outputs/_trip_2.dat',sep='\t',header='infer')

In [6]:
# trip['hhno']=trip['hhno'].astype(str)
# trip['pno']=trip['pno'].astype(str)

# trip['person_id']=trip['hhno']+'-'+trip['pno']

# trip_ei_person=trip[(trip['otaz']<1000)|(trip['dtaz']<1000)]
# trip_ei_person=trip_ei_person[['person_id','hhno']]

# trip_int_int_person=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
# trip_int_int_person=trip_int_int_person[['person_id','hhno']]
# trip_int_int_person.head()

In [4]:
trip_ei_person=pd.read_csv('agend_id.csv.gz')
trip_ei_person.head()

,hhno_x,pno_x,person_id,vot,hhvehs,mode,hhwkrs,hhincome,pagey,pgend
0,1,1,1-1,2.354249,0,walk_transit,0,942,55,female
1,1,1,1-1,2.354249,0,walk_transit,0,942,55,female
2,10,1,10-1,15.378994,0,bike,1,28888,51,male
3,10,1,10-1,15.378994,0,bike,1,28888,51,male
4,100,1,100-1,19.846096,3,car,4,48612,59,male


In [6]:
100*13000/1480041

0.8783540455973855

In [5]:
len(trip_ei_person['person_id'].unique())

1480041

In [7]:
person_ei['ext']=person_ei['person_id'].apply(lambda x: x[0:2])

person_ei

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext
0,63,63-3,female,30,38016,auto_sufficient,7.810582,63
1,63,63-7,male,9,38016,auto_sufficient,7.389009,63
2,131,131-2,male,31,29512,auto_deficient,7.402262,13
3,320,320-1,female,37,108069,auto_deficient,24.262024,32
4,320,ix-320-2,male,37,108069,auto_deficient,12.795260,ix
...,...,...,...,...,...,...,...,...
12797,2809200,2809200-1,female,20,1407,no_auto,1.086607,28
12798,2815778,ix-2815778-1,female,19,14033,auto_sufficient,2.955470,ix
12799,2816020,ix-2816020-1,male,17,16365,auto_sufficient,2.406675,ix
12800,2817726,2817726-1,female,55,6933,auto_sufficient,1.686839,28


In [9]:
person_ei[person_ei['person_id']=='1-1']

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext


In [9]:
# def ext_id(df):
    
#     if df['ext']=='ix':
#         return df['person_id'][2:]
#     else:
#         return df['person_id']
    
# person_ei['agent_id']=person_ei.apply(ext_id,axis=1)
# person_ei.head()

In [77]:
person_ei=person_ei.drop_duplicates()
person_ei[person_ei['ext']=='ix']

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext
4,320,ix-320-2,male,37,108069,auto_deficient,21.325434,ix
6,371,ix-371-1,male,59,48724,auto_deficient,11.225571,ix
9,378,ix-378-4,female,59,59052,auto_sufficient,18.248146,ix
12,409,ix-409-2,female,50,118278,auto_deficient,11.277603,ix
16,441,ix-441-3,female,40,63923,auto_deficient,15.772481,ix
...,...,...,...,...,...,...,...,...
12793,2799472,ix-2799472-1,female,19,11058,auto_deficient,4.494027,ix
12795,2801846,ix-2801846-1,male,23,31094,auto_sufficient,13.722934,ix
12796,2807649,ix-2807649-1,female,60,4827,auto_deficient,4.284778,ix
12798,2815778,ix-2815778-1,female,19,14033,auto_sufficient,4.925783,ix


In [12]:
# person['hhno']=person['hhno'].astype(str)
# person['pno']=person['pno'].astype(str)

# person['person_id']=person['hhno']+'-'+person['pno']

In [14]:
households.head()

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,hhhsc,hh515,hhcu5,hhincome,hownrent,hrestype,hhparcel,zone_id,hhtaz,hhexpfac,samptype
3,9,0.04796,1,1,1,1,0,0,0,0,0,0,0,30020,11117,3,720,0,1,1,11
4,5,0.04796,1,1,0,0,0,1,0,0,0,0,0,18373,11117,2,720,0,1,1,11
5,15,0.04796,2,1,0,0,0,2,0,0,0,0,0,9165,11488,3,20,0,1,1,11
6,20,0.04796,2,1,1,1,0,0,1,0,0,0,0,36487,11117,1,720,0,1,1,11
7,1,0.04796,1,0,0,0,0,0,1,0,0,0,0,942,11117,2,720,0,1,1,11


In [21]:
# person_ei=person[person['person_id'].isin(trip_ei_person.agent_id.unique())]

In [10]:
person_ei.head()

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext
0,63,63-3,female,30,38016,auto_sufficient,7.810582,63
1,63,63-7,male,9,38016,auto_sufficient,7.389009,63
2,131,131-2,male,31,29512,auto_deficient,7.402262,13
3,320,320-1,female,37,108069,auto_deficient,24.262024,32
4,320,ix-320-2,male,37,108069,auto_deficient,12.795260,ix


In [15]:
# trip_ei_person=trip_ei_person.drop_duplicates(['person_id','ext'])
# trip_ei_person[trip_ei_person.duplicated('person_id')]

In [16]:
# external_agents=trip_ei_person[trip_ei_person['ext']=='wc']
# external_agents=external_agents.drop_duplicates(['person_id'])
# external_agents.head()

In [17]:
# pop_df=population(households,person_ei)
# pop_df.head()

In [43]:
pop_df_agents[pop_df_agents['person_id_x']=='21-1']

,person_id_x,hhno,pagey,pgend,hhvehs,person_id_y,ext,agent_id,_merge
10,21-1,21,63,1,2,wc21-1,wc,21-1,both


In [18]:
# pop_df_agents=pd.merge(pop_df,external_agents,left_on=['person_id'],right_on=['agent_id'],
#                       how='left',indicator=True)

# pop_df_agents.head()

In [19]:
# pop_df_agents[pop_df_agents['ext']=='wc']

In [20]:
# pop_df_agents['person_id']=np.where(pop_df_agents['ext']=='wc',
#                                     pop_df_agents['person_id_y'],
#                                     pop_df_agents['person_id_x']
#                                    )
# pop_df_agents.tail()

In [21]:

# pop_df_agents['gender']=np.where(pop_df_agents['pgend']==1,
#                                 'male',
#                                 'female')
# pop_df_agents['pagey']=pop_df_agents['pagey'].astype(str)


In [21]:
person_ei[(person_ei['ext']=='ix')&(person_ei['vehicles']=='no_auto')]

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext


In [11]:
def excluded_modes(df):
    
    if df['vehicles']=='no_auto':
        return 'car'
    else:
        if df['ext']=='ix':
            return 'walk,bike'
        else:
            return ''

In [12]:
# person_ei=person_ei.rename(columns={'hhno_x':'hhno',
#                                    'pno_x':'pno'})




person_ei['excluded_mode']=person_ei.apply(excluded_modes,axis=1)
person_ei['excluded_mode'].value_counts()

             7670
walk,bike    3658
car          1474
Name: excluded_mode, dtype: int64

In [13]:
person_ei['VOT']=person_ei['VOT'].astype(str)
person_ei['pagey']=person_ei['pagey'].astype(str)

In [14]:
person_ei.head()

,hhno,person_id,pgend,pagey,hhincome,vehicles,VOT,ext,excluded_mode
0,63,63-3,female,30,38016,auto_sufficient,7.810581828754746,63,
1,63,63-7,male,9,38016,auto_sufficient,7.3890094178218675,63,
2,131,131-2,male,31,29512,auto_deficient,7.402261864551569,13,
3,320,320-1,female,37,108069,auto_deficient,24.26202366335385,32,
4,320,ix-320-2,male,37,108069,auto_deficient,12.795260484570358,ix,"walk,bike"


In [15]:
def create_xml_pop(df_all,size):
    
    root = ET.Element('objectAttributes')

    for i in range(0,len(df_all)+1,size):
    
        j=i+size

        test_df=df_all.iloc[i:j].reset_index()

#         parser = ET.XMLParser(remove_blank_text=True)

#         template = ET.parse('../SF_CHAMP_Converted/persons_attribute.xml',parser)

        #root = template.getroot()
        
        for person in test_df.person_id.unique():
            
            new_df=test_df[test_df['person_id']==person]
            
            person_tag = ET.SubElement(root, 'object') 
            person_tag.set('id',person)


            age_tag = ET.SubElement(person_tag, 'attribute')
            age_tag.set('name','age')
            age_tag.set('class','java.lang.Integer')
            age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]

            sex_tag = ET.SubElement(person_tag, 'attribute')
            sex_tag.set('name','sex')
            sex_tag.set('class','java.lang.String')
            sex_tag.text=new_df.values[0,new_df.columns.get_loc('pgend')]
            
            vot_tag = ET.SubElement(person_tag, 'attribute')
            vot_tag.set('name','valueOfTime')
            vot_tag.set('class','java.lang.Double')
            vot_tag.text=new_df.values[0,new_df.columns.get_loc('VOT')]

            modes_tag = ET.SubElement(person_tag, 'attribute')
            modes_tag.set('name','excluded-modes')
            modes_tag.set('class','java.lang.String')
            modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]

            root.append(person_tag)
        
#         for row in range(0,size):
            

#             person_tag = ET.SubElement(root, 'object') 
#             person_tag.set('id',test_df.values[row,test_df.columns.get_loc('person_id')])


#             age_tag = ET.SubElement(person_tag, 'attribute')
#             age_tag.set('name','age')
#             age_tag.set('class','java.lang.Integer')
#             age_tag.text=test_df.values[row,test_df.columns.get_loc('pagey')]

#             sex_tag = ET.SubElement(person_tag, 'attribute')
#             sex_tag.set('name','sex')
#             sex_tag.set('class','java.lang.String')
#             sex_tag.text=test_df.values[row,test_df.columns.get_loc('gender')]

#             modes_tag = ET.SubElement(person_tag, 'attribute')
#             modes_tag.set('name','excluded-modes')
#             modes_tag.set('class','java.lang.String')
#             modes_tag.text=test_df.values[row,test_df.columns.get_loc('excluded_mode')]

#             root.append(person_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/persons_attribute_update_test.xml', pretty_print=True, xml_declaration=True,
                            doctype='<!DOCTYPE objectAttributes SYSTEM "http://matsim.org/files/dtd/objectattributes_v1.dtd">',
                   encoding="utf-8")

In [16]:
%%time
create_xml_pop(person_ei,15000)

Wall time: 29.1 s


In [85]:
households=households[households['hhno'].isin(person_ei.hhno.unique())]

In [86]:
households

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,hhhsc,hh515,hhcu5,hhincome,hownrent,hrestype,hhparcel,zone_id,hhtaz,hhexpfac,samptype
121,131,0.04796,4,2,2,1,1,0,2,0,0,0,0,29512,11117,2,720,0,1,1,11
237,63,0.04796,11,3,2,2,0,0,5,0,0,4,0,38016,11117,1,720,0,1,1,11
267,320,0.00000,4,2,2,2,0,1,1,0,0,0,0,108069,21003,1,1939,1,2,1,11
323,383,0.00000,5,2,1,1,0,0,1,0,1,2,0,55465,21289,1,1511,1,2,1,11
358,378,0.00000,5,4,3,2,1,0,2,0,0,0,0,59052,21068,2,1461,1,2,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2817084,2816020,0.00000,1,1,0,0,0,0,0,1,0,0,0,16365,63340,5,310517,1809,2019,1,13
2818926,2764384,0.00504,1,1,0,0,0,0,1,0,0,0,0,11227,39191,5,2232,916,917,1,13
2830190,2836680,0.00477,1,1,1,1,0,0,0,0,0,0,0,65272,5225,5,814500,2234,2444,1,13
2831763,2815778,0.00000,1,1,0,0,0,0,0,1,0,0,0,14033,63340,5,310517,1809,2019,1,13


In [87]:
person_ei['hhno']=person_ei['hhno'].astype(str)
households['hhno']=households['hhno'].astype(str)

In [88]:
df=household(households,person_ei[['hhno','person_id']],mzone)


In [89]:
df.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,100050,100050-3,2,31149,552242.437167,4.179782e+06
1,100065,100065-1,0,37240,552341.233348,4.179793e+06
2,100255,100255-1,2,65749,548544.277019,4.179740e+06
3,100317,100317-2,1,108618,548451.389805,4.179553e+06
4,100326,100326-2,1,118028,548436.477693,4.179637e+06


In [90]:
bikes=pd.read_csv('../SF_CHAMP_Converted/bike.csv.gz',compression='gzip')
bikes=bikes.rename(columns={'hhno_x':'hhno','mode':'number_Bikes'})
bikes['hhno']=bikes['hhno'].astype(str)
bikes.head()

,hhno,number_Bikes
0,10,2
1,1000,2
2,100001,2
3,100014,7
4,100018,4


In [91]:
df_bikes=pd.merge(df,bikes,on='hhno',how='left')
df_bikes['number_Bikes']=df_bikes['number_Bikes'].fillna(0)
df_bikes.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,number_Bikes
0,100050,100050-3,2,31149,552242.437167,4.179782e+06,0.0
1,100065,100065-1,0,37240,552341.233348,4.179793e+06,0.0
2,100255,100255-1,2,65749,548544.277019,4.179740e+06,0.0
3,100317,100317-2,1,108618,548451.389805,4.179553e+06,0.0
4,100326,100326-2,1,118028,548436.477693,4.179637e+06,0.0


In [92]:
df_bikes=df_bikes.drop_duplicates()
df_bikes.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,number_Bikes
0,100050,100050-3,2,31149,552242.437167,4.179782e+06,0.0
1,100065,100065-1,0,37240,552341.233348,4.179793e+06,0.0
2,100255,100255-1,2,65749,548544.277019,4.179740e+06,0.0
3,100317,100317-2,1,108618,548451.389805,4.179553e+06,0.0
4,100326,100326-2,1,118028,548436.477693,4.179637e+06,0.0


In [93]:
df_bikes['number_Bikes']=df_bikes['number_Bikes'].astype(int)

In [24]:
x=[]
len(x)

0

In [94]:
def create_xml(hh_pop_mzone,size):
    
    attr_qname = ET.QName("http://www.w3.org/2001/XMLSchema-instance", "schemaLocation")
    nsmap = {None: "http://www.matsim.org/files/dtd",
         "xsi": "http://www.w3.org/2001/XMLSchema-instance"}
    root = ET.Element('households',
                     {attr_qname:'http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/households_v1.0.xsd'},
                     nsmap=nsmap)

    for i in range(0,len(hh_pop_mzone.hhno.unique())+1,size):
    
        j=i+size

        test_df=hh_pop_mzone[hh_pop_mzone['hhno'].isin(hh_pop_mzone.hhno.unique()[i:j])]


        

        for i in test_df['hhno'].unique():

            hh_id=str(i)

            household_tag = ET.SubElement(root, 'household') 
            household_tag.set('id',str(hh_id))


            members_tag = ET.SubElement(household_tag, 'members')

            df=test_df[test_df['hhno']==i]

            
            for person in df.person_id.unique():

                person_tag=ET.SubElement(members_tag,'personId')

                person_tag.set('refId',str(person))



            veh_list=[]
            
            if df.hhvehs.max()>0:
                x=df.hhvehs.max()
                

                for item in range(0,x):
                    vehicle_id='auto-'+hh_id+'-'+str(item)
                    veh_list.append(vehicle_id)
            if df.number_Bikes.max()>0:  
                y=df.number_Bikes.max()
                for item in range(0,y):
                    vehicle_id='bike-'+hh_id+'-'+str(item)
                    veh_list.append(vehicle_id)

            if len(veh_list)>0:
                vehicle_tag=ET.SubElement(household_tag, 'vehicles')
                for vehicles in veh_list:
                    
                    vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
                    vehicle_list_tag.set('refId',str(vehicles))
   

            income_tag=ET.SubElement(household_tag,'income')
            income_tag.set('period','year')
            income_tag.set('currency','usd')
            income_tag.text=str(df.hhincome.max())

            root.append(household_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/household_test.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [34]:
df_bikes[df_bikes['hhno']=='117113']

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,number_Bikes
92033,117113,117113-1,2,112267,549586.679509,4.180239e+06,0
92035,117113,117113-2,2,112267,549586.679509,4.180239e+06,0


In [95]:
%%time
create_xml(df_bikes,15000)

Wall time: 29.2 s


In [69]:
# def create_xml(hh_pop_mzone,size):

#     for i in range(0,len(hh_pop_mzone.hhno.unique())+1,size):
    
#         j=i+size

#         test_df=hh_pop_mzone[hh_pop_mzone['hhno'].isin(hh_pop_mzone.hhno.unique()[i:j])]

#         parser = ET.XMLParser(remove_blank_text=True)

#         template = ET.parse('../SF_CHAMP_Converted/household.xml',parser)

#         root = template.getroot()

#         for i in test_df['hhno'].unique():

#             hh_id=str(i)

#             household_tag = ET.SubElement(root, 'household') 
#             household_tag.set('id',str(hh_id))


#             members_tag = ET.SubElement(household_tag, 'members')

#             df=test_df[test_df['hhno']==i]

            
#             for person in df.person_id.unique():

#                 person_tag=ET.SubElement(members_tag,'personId')

#                 person_tag.set('refId',str(person))



#             veh_list=[]
#             if df.hhvehs.max()>0:
#                 x=df.hhvehs.max()
#                 vehicle_tag=ET.SubElement(household_tag, 'vehicles')

#                 for item in range(0,x):
#                     vehicle_id='auto-'+hh_id+'-'+str(item)
#                     veh_list.append(vehicle_id)
                
#                 y=df.number_Bikes
#                 for item in range(0,x):
#                     vehicle_id='bike-'+hh_id+'-'+str(item)
#                     veh_list.append(vehicle_id)

#                 for vehicles in veh_list:

#                     vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
#                     vehicle_list_tag.set('refId',str(vehicles))
                    
#             bike_list=[]
#             if df.number_Bikes>0:
                
#                 #vehicle_tag=ET.SubElement(household_tag, 'vehicles')

                

#                 for vehicles in veh_list:

#                     vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
#                     vehicle_list_tag.set('refId',str(vehicles))

#             income_tag=ET.SubElement(household_tag,'income')
#             income_tag.set('period','year')
#             income_tag.set('currency','usd')
#             income_tag.text=str(df.hhincome.max())

#             root.append(household_tag)


#         tree=ET.ElementTree(root)
#         tree.write('../SF_CHAMP_Converted/household.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [15]:
df[df['hhno']=='456251']

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
940890,456251,456251-1,4,122467,547275.302602,4.160566e+06


In [96]:
%%time
create_xml(df,15000)

AttributeError: 'DataFrame' object has no attribute 'number_Bikes'

In [99]:
df_hh=df_bikes.drop_duplicates(['hhno'])
df_hh.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,number_Bikes
0,100050,100050-3,2,31149,552242.437167,4.179782e+06,0
1,100065,100065-1,0,37240,552341.233348,4.179793e+06,0
2,100255,100255-1,2,65749,548544.277019,4.179740e+06,0
3,100317,100317-2,1,108618,548451.389805,4.179553e+06,0
4,100326,100326-2,1,118028,548436.477693,4.179637e+06,0


In [100]:
len(df_hh)

9812

In [113]:
def create_xml_attributes(hh_pop_mzone,size):
    
    root=ET.Element('objectattributes')

    for i in range(0,len(hh_pop_mzone.hhno.unique())+1,size):
    
        j=i+size

        test_df=hh_pop_mzone[hh_pop_mzone['hhno'].isin(hh_pop_mzone.hhno.unique()[i:j])]


        for i in test_df['hhno'].unique():

            hh_id=str(i)
    
            household_tag = ET.SubElement(root, 'object') 
            household_tag.set('id',str(hh_id))

            subset_df=test_df[test_df['hhno']==i]

            xcoord_tag = ET.SubElement(household_tag, 'attribute')
            xcoord_tag.set('name','homecoordx')
            xcoord_tag.set('class','java.lang.Double')
            xcoord_tag.text=str(subset_df.values[0,subset_df.columns.get_loc('X_COORD')])

            ycoord_tag = ET.SubElement(household_tag, 'attribute')
            ycoord_tag.set('name','homecoordy')
            ycoord_tag.set('class','java.lang.Double')
            ycoord_tag.text=str(subset_df.values[0,subset_df.columns.get_loc('Y_COORD')])

            housing_type_tag = ET.SubElement(household_tag, 'attribute')
            housing_type_tag.set('name','housingtype')
            housing_type_tag.set('class','java.lang.String')
            housing_type_tag.text='House'

        root.append(household_tag)


        tree=ET.ElementTree(root)
        tree.write('../SF_CHAMP_Converted/household_attributes_test.xml', pretty_print=True, xml_declaration=True,   
                   doctype='<!DOCTYPE objectattributes SYSTEM "http://www.matsim.org/files/dtd/objectattributes_v1.dtd">',encoding="utf-8")

In [102]:
%%time
create_xml_attributes(df_hh,15000)

Wall time: 21 s


In [164]:
%%time
parser = ET.XMLParser(remove_blank_text=True)

template = ET.parse('SF_CHAMP_Converted/household_int_int_attributes.xml',parser)

root = template.getroot()

for i in df['hhno'].unique():
    
    hh_id=str(i)
    
    household_tag = ET.SubElement(root, 'object') 
    household_tag.set('id',str(hh_id))

    subset_df=df[df['hhno']==i]
    
    xcoord_tag = ET.SubElement(household_tag, 'attribute')
    xcoord_tag.set('name','homecoordx')
    xcoord_tag.set('class','java.lang.Double')
    xcoord_tag.text=str(subset_df['X_COORD'].unique()[0])
    
    ycoord_tag = ET.SubElement(household_tag, 'attribute')
    ycoord_tag.set('name','homecoordy')
    ycoord_tag.set('class','java.lang.Double')
    ycoord_tag.text=str(subset_df['Y_COORD'].unique()[0])
    
    housing_type_tag = ET.SubElement(household_tag, 'attribute')
    housing_type_tag.set('name','housingtype')
    housing_type_tag.set('class','java.lang.String')
    housing_type_tag.text='House'
    
    root.append(household_tag)
    

    tree=ET.ElementTree(root)
    tree.write('SF_CHAMP_Converted/household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")    

KeyboardInterrupt: 

In [103]:
df_bikes.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,number_Bikes
0,100050,100050-3,2,31149,552242.437167,4.179782e+06,0
1,100065,100065-1,0,37240,552341.233348,4.179793e+06,0
2,100255,100255-1,2,65749,548544.277019,4.179740e+06,0
3,100317,100317-2,1,108618,548451.389805,4.179553e+06,0
4,100326,100326-2,1,118028,548436.477693,4.179637e+06,0


In [104]:
df_bikes_available=df_bikes[df_bikes['number_Bikes']>0]
df_bikes_available=df_bikes_available.drop_duplicates('hhno')
df_bikes_available=df_bikes_available[['hhno','number_Bikes']]
df_bikes_available.head()

,hhno,number_Bikes
14,100516,5
26,101134,3
37,101327,6
44,101835,1
47,101843,3


In [105]:
def bike_id(df):
    hh_id=df['hhno']
    number=df['number_Bikes']
    list=[]
    for item in range (0,number):
        list.append('bike-'+hh_id+'-'+str(item))
    return list
def veh_id(df):
    hh_id=df['hhno']
    number=df['hhvehs']
    list=[]
    for item in range (0,number):
        list.append('auto-'+hh_id+'-'+str(item))
    return list
df_bikes_available['vehicle_id']=df_bikes_available.apply(bike_id,axis=1)
df_bikes_available.head()

,hhno,number_Bikes,vehicle_id
14,100516,5,"[bike-100516-0, bike-100516-1, bike-100516-2, ..."
26,101134,3,"[bike-101134-0, bike-101134-1, bike-101134-2]"
37,101327,6,"[bike-101327-0, bike-101327-1, bike-101327-2, ..."
44,101835,1,[bike-101835-0]
47,101843,3,"[bike-101843-0, bike-101843-1, bike-101843-2]"


In [106]:
df_bikes_available=df_bikes_available.explode('vehicle_id')
df_bikes_available[df_bikes_available['hhno']=='100014']

,hhno,number_Bikes,vehicle_id


In [107]:
df_vehs_available=df_bikes[df_bikes['hhvehs']>0]
df_vehs_available=df_vehs_available.drop_duplicates('hhno')
df_vehs_available=df_vehs_available[['hhno','hhvehs']]
df_vehs_available.head()

,hhno,hhvehs
0,100050,2
2,100255,2
3,100317,1
4,100326,1
6,10038,4


In [108]:
df_vehs_available['vehicle_id']=df_vehs_available.apply(veh_id,axis=1)
df_vehs_available.head()

,hhno,hhvehs,vehicle_id
0,100050,2,"[auto-100050-0, auto-100050-1]"
2,100255,2,"[auto-100255-0, auto-100255-1]"
3,100317,1,[auto-100317-0]
4,100326,1,[auto-100326-0]
6,10038,4,"[auto-10038-0, auto-10038-1, auto-10038-2, aut..."


In [109]:
df_vehs_available=df_vehs_available.explode('vehicle_id')
df_vehs_available[df_vehs_available['hhno']=='100014']

,hhno,hhvehs,vehicle_id


In [110]:
df_vehs_available.drop(columns=['hhvehs'],inplace=True)
df_bikes_available.drop(columns=['number_Bikes'],inplace=True)

df_vehs_available['vehicleTypeId']='Car'
df_bikes_available['vehicleTypeId']='BIKE-DEFAULT'

df_all_vehs=pd.concat([df_vehs_available,df_bikes_available])
df_all_vehs.head()

,hhno,vehicle_id,vehicleTypeId
0,100050,auto-100050-0,Car
0,100050,auto-100050-1,Car
2,100255,auto-100255-0,Car
2,100255,auto-100255-1,Car
3,100317,auto-100317-0,Car


In [111]:
df_all_vehs=df_all_vehs.rename(columns={'vehicle_id':'vehicleId'})
df_all_vehs

,hhno,vehicleId,vehicleTypeId
0,100050,auto-100050-0,Car
0,100050,auto-100050-1,Car
2,100255,auto-100255-0,Car
2,100255,auto-100255-1,Car
3,100317,auto-100317-0,Car
...,...,...,...
12765,99178,bike-99178-0,BIKE-DEFAULT
12782,99709,bike-99709-0,BIKE-DEFAULT
12782,99709,bike-99709-1,BIKE-DEFAULT
12782,99709,bike-99709-2,BIKE-DEFAULT


In [112]:

df_all_vehs[['vehicleId','vehicleTypeId']].to_csv('../SF_CHAMP_Converted/vehicles_test.csv.gz',index=False,compression='gzip')

In [67]:
df=pd.read_csv('../SF_CHAMP_Converted/vehicles_test.csv.gz',compression='gzip')
df.head()

,vehicle_id,vehicleTypeId
0,auto-131-0,Car
1,auto-131-1,Car
2,auto-320-0,Car
3,auto-320-1,Car
4,auto-63-0,Car


In [5]:
df.vehicleTypeId.unique()

array(['Car', 'BIKE-DEFAULT'], dtype=object)

In [71]:
%%time
df_w_veh=df_bikes[df_bikes['hhvehs']>0]

df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'1',
                  np.nan)
df_w_veh['veh_3']=np.where(df_w_veh['hhvehs']>2,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'2',
                  np.nan)
df_w_veh['veh_4']=np.where(df_w_veh['hhvehs']>3,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'3',
                  np.nan)
vehs=df_w_veh[['veh_1','veh_2','veh_3','veh_4']]
vehicle_1=vehs[['veh_1']]
vehicle_1=vehicle_1.rename(columns={'veh_1':'vehicleId'})
vehicle_2=vehs[vehs['veh_2'].notnull()][['veh_2']]
vehicle_2=vehicle_2.rename(columns={'veh_2':'vehicleId'})
vehicle_3=vehs[vehs['veh_3'].notnull()][['veh_3']]
vehicle_3=vehicle_3.rename(columns={'veh_3':'vehicleId'})
vehicle_4=vehs[vehs['veh_4'].notnull()][['veh_4']]
vehicle_4=vehicle_4.rename(columns={'veh_4':'vehicleId'})

df_1=pd.concat([vehicle_1,vehicle_2,vehicle_3,vehicle_4])
df_1['vehicleTypeId']='Car'
df_1.to_csv('../SF_CHAMP_Converted/vehicles.csv.gz',index=False,compression='gzip')

<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

Wall time: 15.8 s


In [172]:
def create_xml_pop_template(df):

    root = ET.Element('personsAttributes') 

    for i in df['person_id'].unique():
        
        new_df=df[df['person_id']==i]


        age_tag = ET.SubElement(person_tag, 'attribute')
        age_tag.set('name','age')
        age_tag.set('class','java.lang.Integer')
        age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]
        #age_tag.set('age',df.values[0,df.columns.get_loc('pagey')])
        
        sex_tag = ET.SubElement(person_tag, 'attribute')
        sex_tag.set('name','sex')
        sex_tag.set('class','java.lang.String')
        sex_tag.text=new_df.values[0,new_df.columns.get_loc('gender')]
        #sex_tag.set('sex',df.values[0,df.columns.get_loc('gender')])
        
        modes_tag = ET.SubElement(person_tag, 'attribute')
        modes_tag.set('name','excluded-modes')
        modes_tag.set('class','java.lang.String')
        modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]
        #modes_tag.text=np.where(new_df['excluded_mode'][0]=='0','',new_df['excluded_mode'][0])
#         if new_df['excluded_mode'][0]=='0':
#             modes_tag.text=''
#         else:
#             modes_tag.text=new_df['excluded_mode'][0]
        #modes_tag.set('excluded-modes',df.values[0,df.columns.get_loc('excluded_mode')])


        root.append(person_tag)


    tree=ET.ElementTree(root)
    tree.write('persons_attribute_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [215]:
create_xml_pop_template(pop_df.head(1))

In [208]:
for i in range(1,21,5):
    
        print(i,i+5)

1 6
6 11
11 16
16 21


In [199]:
test=pop_df.iloc[5:10].reset_index()
test

,index,person_id,hhno,pagey,pgend,hhvehs,excluded_mode,gender
0,5,11-1,11,68,2,1,nan,female
1,6,10-1,10,51,1,0,car,male
2,7,3-1,3,47,1,0,car,male
3,8,31-1,31,60,1,1,nan,male
4,9,21-1,21,63,1,2,nan,male


In [205]:
test.values[4,test.columns.get_loc('pagey')]

'63'

In [211]:
def create_xml_pop(df_all,size):

    for i in range(1,len(df_all)+1,size):
    
        j=i+size

        test_df=df_all.iloc[i:j].reset_index()

        parser = ET.XMLParser(remove_blank_text=True)

        template = ET.parse('persons_attribute_int_int.xml',parser)

        root = template.getroot()
        
        for row in range(0,size-1):

            new_df=df[df['person_id']==i]


            age_tag = ET.SubElement(person_tag, 'attribute')
            age_tag.set('name','age')
            age_tag.set('class','java.lang.Integer')
            age_tag.text=new_df.values[0,new_df.columns.get_loc('pagey')]
            #age_tag.set('age',df.values[0,df.columns.get_loc('pagey')])

            sex_tag = ET.SubElement(person_tag, 'attribute')
            sex_tag.set('name','sex')
            sex_tag.set('class','java.lang.String')
            sex_tag.text=new_df.values[0,new_df.columns.get_loc('gender')]
            #sex_tag.set('sex',df.values[0,df.columns.get_loc('gender')])

            modes_tag = ET.SubElement(person_tag, 'attribute')
            modes_tag.set('name','excluded-modes')
            modes_tag.set('class','java.lang.String')
            modes_tag.text=new_df.values[0,new_df.columns.get_loc('excluded_mode')]
            #modes_tag.text=np.where(new_df['excluded_mode'][0]=='0','',new_df['excluded_mode'][0])
    #         if new_df['excluded_mode'][0]=='0':
    #             modes_tag.text=''
    #         else:
    #             modes_tag.text=new_df['excluded_mode'][0]
            #modes_tag.set('excluded-modes',df.values[0,df.columns.get_loc('excluded_mode')])


            root.append(person_tag)


        tree=ET.ElementTree(root)
        tree.write('persons_attribute_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [216]:
%%time
create_xml_pop(pop_df,5000)

IndexError: index 11 is out of bounds for axis 0 with size 11

In [175]:
%%time
create_xml_pop_template(pop_df)

KeyboardInterrupt: 

In [ ]:
veh_list=[]
if df.hhvehs.max()>0:
    x=df.hhvehs.max()

    for item in range(0,x):
        vehicle_id='auto-'+hh_id+'-'+str(item)
        veh_list.append(vehicle_id)

    for vehicles in veh_list:
        vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
        vehicle_tag.set('refId',str(vehicles))

In [ ]:
households_int_int=households[households['hhno'].isin(pop_df.hhno.unique())]

In [ ]:
df=household(households_int_int,person_int_int,mzone)

In [230]:
from lxml import etree as ET
def create_xml_test(hh_pop_mzone):

    root = ET.Element('households') 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personID')
            
            person_tag.set('refID',str(person))
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
                vehicle_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_test.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [231]:
create_xml_test(df_all.head(10))

In [235]:
from lxml import objectify

xml_data = objectify.parse('household_test.xml')  # Parse XML data
root = xml_data.getroot()  # Root element

data = []
cols=[]

for i in range(len(root.getchildren())):
    child = root.getchildren()[i]
    data.append([subchild.text for subchild in child.getchildren()])
    cols.append(child.tag)
    
df = pd.DataFrame(data)  # Create DataFrame and transpose it

In [252]:
root.findall('vehicleDefinitionID')

[]

In [255]:
df_w_veh=df_all[df_all['hhvehs']>0]
df_w_veh.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06
6,8,8-1,1,31668,551662.441744,4.173739e+06
7,9,9-1,1,30020,551662.441744,4.173739e+06
9,11,11-1,1,36157,551662.441744,4.173739e+06


In [256]:
df_w_veh.dtypes

hhno          object
person_id     object
hhvehs         int64
hhincome       int64
X_COORD      float64
Y_COORD      float64
dtype: object

In [261]:
df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'1',
                  np.nan)
df_w_veh['veh_3']=np.where(df_w_veh['hhvehs']>2,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'2',
                  np.nan)
df_w_veh['veh_4']=np.where(df_w_veh['hhvehs']>3,
                   'auto'+'-'+df_w_veh['hhno']+'-'+'3',
                  np.nan)

df_w_veh.head()

<ipython-input-261-a6874fb97696>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_veh['veh_1']='auto'+'-'+df_w_veh['hhno']+'-'+'0'
<ipython-input-261-a6874fb97696>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w_veh['veh_2']=np.where(df_w_veh['hhvehs']>1,
<ipython-input-261-a6874fb97696>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD,veh_1,veh_2,veh_3,veh_4
3,5,5-1,1,18373,551662.441744,4.173739e+06,auto-5-0,NaN,NaN,NaN
4,6,6-1,1,16311,551814.520039,4.173807e+06,auto-6-0,NaN,NaN,NaN
6,8,8-1,1,31668,551662.441744,4.173739e+06,auto-8-0,NaN,NaN,NaN
7,9,9-1,1,30020,551662.441744,4.173739e+06,auto-9-0,NaN,NaN,NaN
9,11,11-1,1,36157,551662.441744,4.173739e+06,auto-11-0,NaN,NaN,NaN


In [262]:
vehs=df_w_veh[['veh_1','veh_2','veh_3','veh_4']]
vehs.head()  

,veh_1,veh_2,veh_3,veh_4
3,auto-5-0,NaN,NaN,NaN
4,auto-6-0,NaN,NaN,NaN
6,auto-8-0,NaN,NaN,NaN
7,auto-9-0,NaN,NaN,NaN
9,auto-11-0,NaN,NaN,NaN


In [268]:
vehicle_1=vehs[['veh_1']]
vehicle_1=vehicle_1.rename(columns={'veh_1':'vehicle_id'})
vehicle_2=vehs[vehs['veh_2'].notnull()][['veh_2']]
vehicle_2=vehicle_2.rename(columns={'veh_2':'vehicle_id'})
vehicle_3=vehs[vehs['veh_3'].notnull()][['veh_3']]
vehicle_3=vehicle_3.rename(columns={'veh_3':'vehicle_id'})
vehicle_4=vehs[vehs['veh_4'].notnull()][['veh_4']]
vehicle_4=vehicle_4.rename(columns={'veh_4':'vehicle_id'})
vehicle_1.head()

,vehicle_id
3,auto-5-0
4,auto-6-0
6,auto-8-0
7,auto-9-0
9,auto-11-0


In [271]:
df_1=pd.concat([vehicle_1,vehicle_2,vehicle_3,vehicle_4])
df_1['vehicleTypeID']='Car'
df_1.to_csv('vehicles.csv')

In [59]:
trip=pd.read_table('_trip_2.dat',sep='\t',header='infer')
trip.head()

,id,tour_id,hhno,pno,day,tour,half,tseg,tsvid,opurp,dpurp,oadtyp,dadtyp,opcl,otaz,dpcl,dtaz,mode,pathtype,dorp,deptm,arrtm,endacttm,travtime,travcost,travdist,vot,trexpfac
0,9101,91,9,1,1,1,1,1,0,0,1,1,2,720,1,2673,440,3,2,1,419,435,934,16.59,0.5652,4.71,10.980627,1
1,9151,91,9,1,1,1,2,1,0,1,0,2,1,2673,440,720,1,3,2,1,934,950,179,16.67,0.5604,4.67,10.980627,1
2,5101,51,5,1,1,1,1,1,0,0,5,1,4,720,1,2239,924,6,3,4,578,610,632,32.07,0.4150,6.07,5.281956,1
3,5151,51,5,1,1,1,2,1,0,5,0,4,1,2239,924,720,1,6,3,4,632,663,179,31.57,0.4150,6.00,5.281956,1
4,18101,181,15,2,1,1,1,1,0,0,5,1,4,20,1,1301,918,3,2,1,902,917,926,15.52,0.5112,4.26,2.950609,1


In [60]:
trip_int_int=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
trip_int_int=trip_int_int[['hhno']]
trip_int_int

,hhno
0,9
1,9
2,5
3,5
4,15
...,...
25042938,2837003
25042939,2837003
25042940,2837003
25044018,2834752


In [136]:
# trip['hhno']=trip['hhno'].astype(str)
# trip['pno']=trip['pno'].astype(str)

trip['person_id']=trip['hhno']+'-'+trip['pno']

trip_int_int_person=trip[(trip['otaz']<1000)&(trip['dtaz']<1000)]
trip_int_int_person=trip_int_int_person[['person_id']]
trip_int_int_person.head()


,person_id
0,9-1
1,9-1
2,5-1
3,5-1
4,15-2


In [61]:
households_int_int=households[households['hhno'].isin(trip_int_int.hhno.unique())]

In [138]:
# person['hhno']=person['hhno'].astype(str)
# person['pno']=person['pno'].astype(str)

person['person_id']=person['hhno']+'-'+person['pno']

person_int_int=person[person['person_id'].isin(trip_int_int_person.person_id.unique())]

In [139]:
person_int_int.head()

,id,hhno,pno,pptyp,pagey,pgend,pwtyp,pwpcl,pwtaz,pwautime,pwaudist,pstyp,pspcl,pstaz,psautime,psaudist,puwmode,puwarrp,puwdepp,ptpass,ppaidprk,pdiary,pproxy,psexpfac,person_id
5,9,9,1,1,54,1,1,2673,440,16.63,4.69,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,9-1
6,5,5,1,3,79,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,5-1
8,18,15,2,3,66,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,1,1,-1,-1,1,15-2
10,28,20,2,4,55,1,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,20-2
11,1,1,1,4,55,2,0,-1,-1,-1.00,-1.00,0,-1,-1,-1.0,-1.0,-1,-1,-1,0,1,-1,-1,1,1-1


In [62]:
households_int_int

,hhno,fraction_with_jobs_outside,hhsize,hhvehs,hhwkrs,hhftw,hhptw,hhret,hhoad,hhuni,hhhsc,hh515,hhcu5,hhincome,hownrent,hrestype,hhparcel,zone_id,hhtaz,hhexpfac,samptype
3,9,0.04796,1,1,1,1,0,0,0,0,0,0,0,30020,11117,3,720,0,1,1,11
4,5,0.04796,1,1,0,0,0,1,0,0,0,0,0,18373,11117,2,720,0,1,1,11
5,15,0.04796,2,1,0,0,0,2,0,0,0,0,0,9165,11488,3,20,0,1,1,11
6,20,0.04796,2,1,1,1,0,0,1,0,0,0,0,36487,11117,1,720,0,1,1,11
7,1,0.04796,1,0,0,0,0,0,1,0,0,0,0,942,11117,2,720,0,1,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2836223,2814016,0.00818,1,1,1,0,0,0,0,1,0,0,0,13578,30305,5,314215,1802,2012,1,13
2836614,2834891,0.00907,1,1,1,0,1,0,0,0,0,0,0,7662,273,5,813205,2208,2418,1,13
2836777,2837003,0.00000,1,1,0,0,0,0,1,0,0,0,0,23457,76,5,813000,2243,2453,1,13
2837159,2834752,0.00340,1,1,1,0,1,0,0,0,0,0,0,7859,9451,5,813523,2202,2412,1,13


In [63]:
df=household(households_int_int,person,mzone)
df.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,1-1,0,942,551662.441744,4.173739e+06
1,3,3-1,0,8728,551814.520039,4.173807e+06
2,4,4-1,0,5052,551662.441744,4.173739e+06
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06


In [11]:
df_1=df.head(1)

In [71]:
df_all=df[df['hhno']!='1']

In [72]:
df_all

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
1,3,3-1,0,8728,551814.520039,4.173807e+06
2,4,4-1,0,5052,551662.441744,4.173739e+06
3,5,5-1,1,18373,551662.441744,4.173739e+06
4,6,6-1,1,16311,551814.520039,4.173807e+06
5,7,7-1,0,11451,551662.441744,4.173739e+06
...,...,...,...,...,...,...
1176194,2837003,2837003-1,1,23457,538448.122055,4.190256e+06
1176195,2837007,2837007-1,0,0,541825.205926,4.187812e+06
1176196,2837035,2837035-1,1,3649,527226.292042,4.195602e+06
1176197,2837166,2837166-1,1,0,527226.292042,4.195602e+06


In [14]:
create_xml(df_1)

In [26]:
%%time
from lxml import etree as ET


for i in range(0,len(df_all)+5000,5000):
    
    j=i-5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[j:i])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int.xml',parser)
    
    root = template.getroot() 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personId')
            
            person_tag.set('refId',str(person))
        
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            vehicle_tag=ET.SubElement(household_tag, 'vehicles')
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                
                vehicle_list_tag = ET.SubElement(vehicle_tag, 'vehicleDefinitionId')
                vehicle_list_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('currency','usd')
        income_tag.text=str(df.hhincome.max())
        #income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('SmallSample/household_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")




Wall time: 40min 33s


In [13]:
from lxml import etree as ET
def create_xml(hh_pop_mzone):

    root = ET.Element('households') 

    for i in hh_pop_mzone['hhno'].unique():
        
        hh_id=str(i)

        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))


        members_tag = ET.SubElement(household_tag, 'members')

        df=hh_pop_mzone[hh_pop_mzone['hhno']==i]
        
        person_list=df.person_id.unique()
        for person in df.person_id.unique():
            
            person_tag=ET.SubElement(members_tag,'personID')
            
            person_tag.set('refID',str(person))
        
        
        veh_list=[]
        if df.hhvehs.max()>0:
            x=df.hhvehs.max()
            
            for item in range(0,x):
                vehicle_id='auto-'+hh_id+'-'+str(item)
                veh_list.append(vehicle_id)
                
            for vehicles in veh_list:
                vehicle_tag = ET.SubElement(household_tag, 'vehicleDefinitionID') 
                vehicle_tag.set('refId',str(vehicles))
                
        income_tag=ET.SubElement(household_tag,'income')
        income_tag.set('period','year')
        income_tag.set('value',str(df.hhincome.max()))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_int_int.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")

In [10]:
create_xml(df)

KeyboardInterrupt: 

In [118]:
df_test=df_all.reset_index()
df_test

,index,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,1,3,3-1,0,8728,551814.520039,4.173807e+06
1,2,4,4-1,0,5052,551662.441744,4.173739e+06
2,3,5,5-1,1,18373,551662.441744,4.173739e+06
3,4,6,6-1,1,16311,551814.520039,4.173807e+06
4,5,7,7-1,0,11451,551662.441744,4.173739e+06
...,...,...,...,...,...,...,...
1176193,1176194,2837003,2837003-1,1,23457,538448.122055,4.190256e+06
1176194,1176195,2837007,2837007-1,0,0,541825.205926,4.187812e+06
1176195,1176196,2837035,2837035-1,1,3649,527226.292042,4.195602e+06
1176196,1176197,2837166,2837166-1,1,0,527226.292042,4.195602e+06


In [127]:
df_test.hhno.unique()[0:10]

array(['3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], dtype=object)

In [126]:
df_all.hhno.unique()

array(['3', '4', '5', ..., '2837035', '2837166', '2837191'], dtype=object)

In [130]:
%%time
from lxml import etree as ET


for i in range(0,len(df_all.hhno.unique())+5000,5000):
    
    j=i+5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[i:j])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int_attributes.xml',parser)
    root = template.getroot()

    for hh in test_df['hhno'].unique():
        
        hh_id=str(hh)
    
        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))

        subset_df=df_all[df_all['hhno']==hh]

        xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
        xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))

        ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
        ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))

        root.append(household_tag)

        


    tree=ET.ElementTree(root)
    tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")


Wall time: 12h 3min 47s


In [116]:
%%time
root=ET.Element('householdAttributes')

for i in df_all['hhno'].unique():
    
    hh_id=str(i)
    
    household_tag = ET.SubElement(root, 'household') 
    household_tag.set('id',str(hh_id))

    subset_df=df_all[df_all['hhno']==i]
    
    xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
    xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))
    
    ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
    ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))
    
    root.append(household_tag)
    

tree=ET.ElementTree(root)
tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")    

KeyboardInterrupt: 

In [42]:

df_all.drop(columns=['level_0'],inplace=True)
df_all.head()

,hhno,person_id,hhvehs,hhincome,X_COORD,Y_COORD
0,3,3-1,0,8728,551814.520039,4.173807e+06
1,4,4-1,0,5052,551662.441744,4.173739e+06
2,5,5-1,1,18373,551662.441744,4.173739e+06
3,6,6-1,1,16311,551814.520039,4.173807e+06
4,7,7-1,0,11451,551662.441744,4.173739e+06


In [101]:
df_all.to_csv('households.csv')

In [113]:
%%time
from lxml import etree as ET

for i in range(0,len(df_all)+5000,5000):
    
    j=i+5000
    
    test_df=df_all[df_all['hhno'].isin(df_all.hhno.unique()[i:j])]

    parser = ET.XMLParser(remove_blank_text=True)

    template = ET.parse('household_int_int_attributes.xml',parser)
    
    root = template.getroot() 

    for hh_unique in test_df['hhno'].unique():
        
        hh_id=str(i)
    
        household_tag = ET.SubElement(root, 'household') 
        household_tag.set('id',str(hh_id))

        subset_df=test_df[test_df['hhno']==i]

        xcoord_tag = ET.SubElement(household_tag, 'homexcoord')
        xcoord_tag.set('x',str(subset_df['X_COORD'].unique()[0]))

        ycoord_tag = ET.SubElement(household_tag, 'homeycoord')
        ycoord_tag.set('y',str(subset_df['Y_COORD'].unique()[0]))

        root.append(household_tag)


    tree=ET.ElementTree(root)
    tree.write('household_int_int_attributes.xml', pretty_print=True, xml_declaration=True,   encoding="utf-8")





IndexError: index 0 is out of bounds for axis 0 with size 0